In [ ]:
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -o annotations_trainval2017.zip

In [ ]:
%pip install transformers

In [ ]:
import re
import json
import pandas as pd
with open('./annotations/captions_val2017.json') as json_file:
    data = json.load(json_file)

df= pd.DataFrame(data['annotations'])

with open('./annotations/captions_train2017.json') as json_file:
    data = json.load(json_file)

df2= pd.DataFrame(data['annotations'])

In [ ]:
df_val_pair=df.copy()
df_val_pair["file_name"]=df_val_pair.apply(lambda x: str(x.image_id).zfill(12)+'.jpg', axis=1)
df_val_pair=df_val_pair.reset_index().set_index("file_name")
df_val_pair=df_val_pair.drop(columns=['index','id','image_id'])
df_val_pair

In [ ]:
df_train_pair=df2.copy()
df_train_pair["file_name"]=df_train_pair.apply(lambda x: str(x.image_id).zfill(12)+'.jpg', axis=1)
df_train_pair=df_train_pair.reset_index().set_index("file_name")
df_train_pair=df_train_pair.drop(columns=['index','id','image_id'])
df_train_pair

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
max_length=255
padding="max_length"
df_train_pair["token_caption"]=df_train_pair.apply(lambda x:tokenizer(x.caption, return_tensors="pt",max_length=max_length,padding=padding)['input_ids'].squeeze().numpy(), axis=1)

In [ ]:
df_val_pair["token_caption"]=df_val_pair.apply(lambda x:tokenizer(x.caption, return_tensors="pt",max_length=max_length,padding=padding)['input_ids'].squeeze().numpy(), axis=1)

In [ ]:
from tqdm import tqdm
import s3fs
import os

S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
S3_BUCKET="cthiounn2"
            
files=['val2017.parquet','train2017.parquet']
for file in tqdm(files):
    with fs.open(f'{S3_BUCKET}/{file}', mode="rb") as file_in, open(file,"wb") as file_out:
            file_out.write(file_in.read())


In [ ]:
df_train2017=pd.read_parquet("train2017.parquet")
df_train2017=df_train2017.reset_index().set_index("file_name")
df_train2017_merge=df_train2017.merge(df_train_pair,how="outer",left_index=True,right_index=True)
df_train2017_merge_extract=df_train2017_merge[["token_caption","encoding"]]
df_train2017_merge_extract.to_parquet("train2017_caption_image.parquet")

In [ ]:
df_val2017=pd.read_parquet("val2017.parquet")
df_val2017=df_val2017.reset_index().set_index("file_name")
df_val2017_merge=df_val2017.merge(df_val_pair,how="outer",left_index=True,right_index=True)
df_val2017_merge_extract=df_val2017_merge[["token_caption","encoding"]]
df_val2017_merge_extract.to_parquet("val2017_caption_image.parquet")

In [ ]:
def write_file_to_s3(bucket_name:str,dir_file:str,file_name:str,fs:s3fs.core.S3FileSystem):
    if bucket_name and file_name and fs:
        FILE_PATH_OUT_S3 = bucket_name + "/" + file_name
        with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out , open(dir_file+file_name, 'rb') as file_in:
            file_out.write(file_in.read())
            
write_file_to_s3(S3_BUCKET,'./','train2017_caption_image.parquet',fs)
write_file_to_s3(S3_BUCKET,'./','val2017_caption_image.parquet',fs)